In [1]:
from sentence_transformers import SentenceTransformer, util
import nltk
from sklearn.cluster import AgglomerativeClustering
import json
from transformers import pipeline, AutoTokenizer, AutoModel
from transformers import GPT2TokenizerFast

/export/home/usmanova/miniconda3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/export/home/usmanova/miniconda3/lib/python3.12/site-packages/transformers/utils/hub.py:128: FutureWarning: Using `PYTORCH_PRETRAINED_BERT_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
/export/home/usmanova/miniconda3/lib/python3.12/site-packages/transformers/utils/hub.py:128: FutureWarning: Using `PYTORCH_TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
/export/home/usmanova/miniconda3/lib/python3.12/site-packages/transformers/utils/hub.py:128: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [84]:
# Remove following from the parsed pdfs
# <!-- image -->
# ; -> ,
# \_

In [2]:
import json

def save_json(path, content):
    with open(path, 'w') as f:
        json.dump(content, f, indent=2)

def read_json(path):
    with open(path) as json_data:
        content = json.load(json_data)
    return content

In [3]:
root = "/storage/usmanova/judgeclaim/reports/processed/"

# Not in ClimRetrieve base
# Apple_2023_EnvironmentalProgress_Report
# BHP Climate Change Report 2020
# BHP Climate Transition Action Plan
# CostCo Climate Action Plan
# PepsiCo ESG 2022
# PWC China ESG Report 2023
# Qantas 2023 Sustainability Report
# Shell sustainability report 2022


orgs = [
    "2022 Microsoft Environmental Sustainability Report",
    "AstraZeneca Sustainability Report 2023",
    "2023 Lloyds sustainability report",
    "AEO 2022 ESG Report",
    # "Apple_2023_EnvironmentalProgress_Report",
    "AT&T 2022 Sustainability Summary",
    # "BHP Climate Change Report 2020",
    # "BHP Climate Transition Action Plan",
    "Boeing 2023 Sustainability Report",
    "Coca-Cola Business Sustainability Report 2022",
    # "CostCo Climate Action Plan",
    "CT REIT 2022 ESG Report",
    "Deloitte 2023 Global Impact Report",
    "HM Group Sustainability Disclosure 2022",
    "HSBC Annual Holdings and Accounts Report 2023",
    "JP Morgan Climate Report 2022",
    "Mastercard 2022 ESG Report",
    "McKinsey 2022 ESG Full Report",
    "Meta 2023 Sustainability Report",
    "Microsoft 2022 Environmental Sustainability Report",
    "Netflix ESG Report 2022",
    "New Look sustainability report",
    "PayPal Global Impact Report 2023",
    # "PepsiCo ESG 2022",
    # "PWC China ESG Report 2023",
    "Qantas 2023 Sustainability Report",
    "Rio Tinto Climate Change Report 2023",
    "Ryanair 2022 Sustainability Report",
    # "Schneider Sustainability Report",
    # "Shell sustainability report 2022",
    "Sony Sustainability Report 2023",
    "Starbucks Environmental & Social Impact Report 2022",
    "Suncor Sustainability Report 2023",
    "University of Oxford 2022 Sustainability Report",
    "Veolia ESG Report 2023",
    "Walmart ESG Highlights 2023",
    "Westpac 2023 climate report",
    "Woolworths Group 2023 Sustainability Report"
]

orgs_code = {
    "2022 Microsoft Environmental Sustainability Report": "microsoft-sustainability-2022",
    "AstraZeneca Sustainability Report 2023": "astrazeneca-sustainability-2023",
    "2023 Lloyds sustainability report": "lloyds-sustainability-2023",
    "AEO 2022 ESG Report": "aeo-esg-2022",
    "Apple_2023_EnvironmentalProgress_Report": "apple-environment-2023",
    "AT&T 2022 Sustainability Summary": "at&t-sustainability-2022",
    "BHP Climate Change Report 2020": "bhp-climate-2020",
    "BHP Climate Transition Action Plan": "bhp-climate-transition-2021",
    "Boeing 2023 Sustainability Report": "boeing-sustainability-2023",
    "Coca-Cola Business Sustainability Report 2022": "cocacola-sustainability-2022",
    "CostCo Climate Action Plan": "costco-climate-action-2024",
    "CT REIT 2022 ESG Report": "ctreit-esg-2022",
    "Deloitte 2023 Global Impact Report": "deloitte-global-2023",
    "HM Group Sustainability Disclosure 2022": "hm-sustainability-2022",
    "HSBC Annual Holdings and Accounts Report 2023": "hsbc-annual-2023",
    "JP Morgan Climate Report 2022": "jpmorgan-climate-2022",
    "Mastercard 2022 ESG Report": "mastercard-esg-2022",
    "McKinsey 2022 ESG Full Report": "mckinsey-esg-2022",
    "Meta 2023 Sustainability Report": "meta-sustainability-2023",
    "Netflix ESG Report 2022": "netflix-esg-2022",
    "New Look sustainability report": "newlook-sustainability-2023",
    "PayPal Global Impact Report 2023": "paypal-global-2023",
    "PepsiCo ESG 2022": "pepsico-esg-2022",
    "PWC China ESG Report 2023": "pwc-china-esg-2023",
    "Qantas 2023 Sustainability Report": "qantas-sustainability-2023",
    "Rio Tinto Climate Change Report 2023": "riotinto-climate-2023",
    "Ryanair 2022 Sustainability Report": "ryanair-sustainability-2022",
    "Schneider Sustainability Report": "schneider-sustainability-2022",
    "Shell sustainability report 2022": "shell-sustainability-2022",
    "Sony Sustainability Report 2023": "sony-sustainability-2023",
    "Starbucks Environmental & Social Impact Report 2022": "starbucks-impact-2022",
    "Suncor Sustainability Report 2023": "suncor-sustainability-2023",
    "University of Oxford 2022 Sustainability Report": "unioxford-sustainability-2022",
    "Veolia ESG Report 2023": "veolia-esg-2023",
    "Walmart ESG Highlights 2023": "walmart-esg-2023",
    "Westpac 2023 climate report": "westpac-climate-2023",
    "Woolworths Group 2023 Sustainability Report": "woolworths-sustainability-2023"
}

In [4]:
# Report to preprocess
IND = 19
org = orgs[IND]+"/"
print("Processing: ", org)

Processing:  Qantas 2023 Sustainability Report/


In [ ]:
# qa_dataset = read_json(root+org+orgs_code[orgs[IND]]+'_final_old.json')
# corpus = read_json(root+org+orgs_code[orgs[IND]]+'.json')

In [ ]:
# for entry in qa_dataset:
#     for par in entry['paragraphs']:
#         for row in corpus:
#             if par['idx'] == row['idx']:
#                 par['text'] = row['text']

In [ ]:
# save_json(root+org+orgs_code[orgs[IND]]+'_final.json', qa_dataset)

### Process corpus

In [5]:
import re

text = open(root+org+"content.txt").read()
p = re.compile("##.*\n")
section_titles = re.findall(p, text)

mod_section_titles = []
for title in section_titles:
    mod_section_titles.append(title[3:len(title)-1])
# mod_section_titles

In [6]:
sections = text.replace('\n', ' ').split("##")
sections.pop(0)
# sections

''

In [7]:
len(sections), len(mod_section_titles)

(202, 202)

In [8]:
# Generate titles and format output
chunked_paragraphs = []
for idx, (section, title) in enumerate(zip(sections, mod_section_titles)):
    # Append paragraph data
    sec_text = section[len(title)+2:]
    if len(sec_text) > 5:
        chunked_paragraphs.append({
            "title": title,
            "text": sec_text,
            "idx": orgs_code[orgs[IND]]+"-"+str(idx)
        })

In [9]:
from nltk.tokenize import sent_tokenize, word_tokenize

def chunk_corpus(corpus: list, chunk_size: int = 64) -> list:
    new_corpus = []
    for p in corpus:
        text = p['text']
        idx = p['idx'] if 'idx' in p else p['_id']
        title = p['title']

        sentences = sent_tokenize(text)
        current_chunk = []
        current_chunk_size = 0

        chunk_idx = 0
        for sentence in sentences:
            words = word_tokenize(sentence)
            if current_chunk_size + len(words) > chunk_size:
                new_corpus.append({
                    **p,
                    'title': title,
                    'text': " ".join(current_chunk),
                    'idx': str(idx) + f"_{chunk_idx}",
                })
                current_chunk = words
                current_chunk_size = len(words)
                chunk_idx += 1
            else:
                current_chunk.extend(words)
                current_chunk_size += len(words)

        if current_chunk:  # there are still some words left
            new_corpus.append({
                **p,
                'title': title,
                'text': " ".join(current_chunk),
                'idx': f"{idx}_{chunk_idx}",
            })

    return new_corpus

In [10]:
new_chunks = chunk_corpus(chunked_paragraphs)

new_chunks_filtered = []
for new_chunk in new_chunks:
    if len(new_chunk['text'].split()) > 4:
        new_chunks_filtered.append(new_chunk)
len(new_chunks_filtered), len(new_chunks), len(chunked_paragraphs)

(549, 575, 202)

In [11]:
save_json(root+org+orgs_code[orgs[IND]]+'_corpus.json', new_chunks_filtered)

### Process ClimRetrieve

In [12]:
import pandas as pd
import numpy as np

climretrieve_df = pd.read_excel("/storage/usmanova/judgeclaim/climretrieve/ClimRetrieve_base.xlsx")
print(climretrieve_df.shape)
climretrieve_df = climretrieve_df.drop(["Unnamed: 0"], axis=1) 
climretrieve_df.head()

(741, 12)


,Document,Question,Context,Relevant,Page,Source From,Answer,Source Relevance Score,Unsure Flag,Addressed Directly,Core 16 Question
0,2022 Microsoft Environmental Sustainability Re...,Do the environmental/sustainability targets se...,We have also engaged and supported the drive t...,In response to the UN Secretary-General's call...,72,text,[YES]. Microsoft's environmental targets refer...,3.0,NaN,directly,1
1,2022 Microsoft Environmental Sustainability Re...,Do the environmental/sustainability targets se...,Microsoft’s actions alone will not solve the c...,"As a global technology leader, we are also com...",65,text,[YES]. Microsoft's environmental targets refer...,3.0,NaN,directly,1
2,2022 Microsoft Environmental Sustainability Re...,Does the company have a strategy on waste mana...,Reducing our waste footprint \nTo meet our com...,To meet our commitment to being zero waste by ...,41,text,[YES]. Microsoft has a strategy on waste manag...,3.0,NaN,directly,1
3,2022 Microsoft Environmental Sustainability Re...,Does the company have a strategy on waste mana...,We recognize the urgent need to reduce carbon ...,To reach our commitment to become a zero waste...,37,text,[YES]. Microsoft has a strategy on waste manag...,3.0,NaN,directly,1
4,2022 Microsoft Environmental Sustainability Re...,Does the company identify any impacts of its b...,Our approach \nMicrosoft directly operates on ...,Microsoft directly operates on approximately 1...,46,text,"[YES]. Microsoft reports their emissions, land...",3.0,NaN,directly,1


In [13]:
# climretrieve_df['Question'].unique()

In [14]:
# filtered = climretrieve_df[climretrieve_df['Document'].str.contains("Coca-Cola")]
filtered = climretrieve_df[climretrieve_df['Answer'].str.contains("Qantas", na=False)]
filtered = filtered[filtered['Source From'] == "text"]
len(filtered)

6

In [16]:
content_json = read_json(root+org+orgs_code[orgs[IND]]+'_corpus.json')
# astrazeneca = read_json(root+"AstraZeneca Sustainability Report 2023/"+"astrazeneca-sustainability-2023.json")
# combined_orgs = microsoft + astrazeneca
content_df = pd.DataFrame.from_dict(content_json, orient='columns')
content_df

,title,text,idx
0,Sustainability Framework,QANTAS SUSTAINABILITY REPORT 2023 This Sustain...,qantas-sustainability-2023-0_0
1,Sustainability Framework,"parties , and external enablers such as techno...",qantas-sustainability-2023-0_1
2,Sustainability Framework,"To the maximum extent permitted by Iaw , Qanta...",qantas-sustainability-2023-0_2
3,Chairman's message,This past year has seen the Qantas Group more ...,qantas-sustainability-2023-1_0
4,Chairman's message,"As our 2023 Sustainability Report details , th...",qantas-sustainability-2023-1_1
...,...,...,...
544,Use of this Assurance Report,This report has been prepared the Directors of...,qantas-sustainability-2023-198_0
545,Use of this Assurance Report,We disclaim any assumption of responsibility f...,qantas-sustainability-2023-198_1
546,Management's responsibility,Management are responsible for : - determining...,qantas-sustainability-2023-199_0
547,Our Responsibility,Our responsibility is to perform a limited ass...,qantas-sustainability-2023-200_0


In [17]:
import string

def check_token_overlap(context, text):
    context_tokens = set(context.split())  # Split context into words
    text_tokens = set(text.split())        # Split text into words
    return len(context_tokens & text_tokens) > 0  # Check for intersection

def preprocess_text(text):
    if pd.isna(text):  # Handle NaN values
        return ""
    # text = text.lower()  # Convert to lowercase
    # text = text.translate(str.maketrans("", "", string.punctuation))  # Remove punctuation
    text = " ".join(text.split())
    return text

def find_paragraph_id(context, paragraphs_df, n=6):
    for _, row in paragraphs_df.iterrows():
        if " ".join(context.split()[:n]) in row['text']:
            return row['idx']
        # if context in row['text']:  # Check if Context is a substring of Text
        #     return row['idx']
    return np.nan

In [18]:
filtered['Relevant'] = filtered['Relevant'].astype(str).apply(preprocess_text)
content_df['text'] = content_df['text'].astype(str).apply(preprocess_text)

# texts = combined_orgs_df['text'].tolist()
# idxs = combined_orgs_df['idx'].tolist()
# filtered['ParagraphID'] = filtered['Relevant'].apply(find_paragraph_id, args=(combined_orgs_df,))

In [19]:
filtered['ParagraphID'] = filtered['Relevant'].apply(find_paragraph_id, args=(content_df,))

In [ ]:
# filtered = filtered[~filtered.Answer.str.contains("CostCo")]
# filtered = filtered[~filtered.Answer.str.contains("AT&T")]
# filtered = filtered.head(14)

In [32]:
filtered

,Document,Question,Context,Relevant,Page,Source From,Answer,Source Relevance Score,Unsure Flag,Addressed Directly,Core 16 Question,ParagraphID
682,Westpac 2023 climate report.pdf,Do the environmental/sustainability targets se...,We recognise that human-induced climate change...,We are committed to supporting the aims of the...,22,text,Yes. Qantas's targets reference external clima...,3.0,NaN,indirectly,1,qantas-sustainability-2023-56_0
683,Westpac 2023 climate report.pdf,Do the environmental/sustainability targets se...,We recognise that human-induced climate change...,We are committed to supporting the aims of the...,22,text,Yes. Qantas's targets aligns with external cli...,3.0,NaN,indirectly,1,qantas-sustainability-2023-56_0
684,Westpac 2023 climate report.pdf,Does the company report short-term actions tak...,We are focused on driving progress across thre...,Operational and fleet efficiency: reducign sin...,23,text,Yes. Qantas report actions planned to reduce w...,3.0,NaN,indirectly,1,qantas-sustainability-2023-61_2
685,Westpac 2023 climate report.pdf,Does the company report short-term actions tak...,"As at 30 June 2023, more than 202.9 million si...",We continue to identify opportunities to reduc...,36,text,Yes. Qantas report actions planned to reduce w...,3.0,NaN,indirectly,1,qantas-sustainability-2023-93_3
689,Westpac 2023 climate report.pdf,Does the company have a strategy on waste mana...,We are focused on driving progress across thre...,Operational and fleet efficiency: reducign sin...,23,text,Yes. Qantas has a strategy to achieve zero gen...,3.0,NaN,indirectly,1,qantas-sustainability-2023-61_2
693,Westpac 2023 climate report.pdf,Does the company have a strategy on waste mana...,We recognise the impact waste has on the envir...,The Group’s ambition is to have zero single-us...,36,text,Yes. Qantas has a strategy to achieve zero gen...,3.0,NaN,indirectly,1,qantas-sustainability-2023-93_0


In [ ]:
# filtered.loc[585] = filtered.loc[570]
# filtered = filtered.drop(207)

In [30]:
# Assign ParagraphID manually
filtered.loc[693, "ParagraphID"] = "qantas-sustainability-2023-93_0"

In [29]:
# Check manually paragraphs
content_df[content_df['text'].str.contains("zero onshore waste to landfill by 2030")]

,title,text,idx
232,Reducing waste through circularity: OUR APPROACH,We recognise the impact waste has on the envir...,qantas-sustainability-2023-93_0
240,WASTE REDUCTION,"We recycled 2,841 tonnes of waste in FY23 , re...",qantas-sustainability-2023-95_0


In [31]:
# Check for Null paragraphs
filtered[filtered.ParagraphID.isnull()]

,Document,Question,Context,Relevant,Page,Source From,Answer,Source Relevance Score,Unsure Flag,Addressed Directly,Core 16 Question,ParagraphID


In [33]:
filtered.to_csv(root+org+orgs_code[orgs[IND]]+'.csv')

In [166]:
# import re

# def check_yes_no(answer):
#     answer_str = str(answer)
#     # Use regular expressions to match 'YES' or 'NO' case-insensitively, ignoring surrounding punctuation
#     if re.search(r'\bYES\b', answer_str, re.IGNORECASE):  # \b ensures it is a whole word match
#         return True
#     elif re.search(r'\bNO\b', answer_str, re.IGNORECASE):
#         return False
#     else:
#         return None  # No match for 'YES' or 'NO'

# # Apply the function to create the new column
# filtered['Supports'] = filtered['Answer'].apply(check_yes_no)
# filtered

/tmp/ipykernel_67550/2803505167.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered['Supports'] = filtered['Answer'].apply(check_yes_no)


,Unnamed: 0,Document,Question,Context,Relevant,Page,Source From,Answer,Source Relevance Score,Unsure Flag,Addressed Directly,Core 16 Question,ParagraphID,Supports
0,0,2022 Microsoft Environmental Sustainability Re...,Do the environmental/sustainability targets se...,we have also engaged and supported the drive t...,in response to the un secretarygenerals call t...,72,text,[YES]. Microsoft's environmental targets refer...,3.0,NaN,directly,1,microsoft-sustainability-2022-256_7,True
1,1,2022 Microsoft Environmental Sustainability Re...,Do the environmental/sustainability targets se...,microsoft’s actions alone will not solve the c...,as a global technology leader we are also comm...,65,text,[YES]. Microsoft's environmental targets refer...,3.0,NaN,directly,1,microsoft-sustainability-2022-20_0,True
2,17,2022 Microsoft Environmental Sustainability Re...,Does the company have a strategy on waste mana...,reducing our waste footprint to meet our commi...,to meet our commitment to being zero waste by ...,41,text,[YES]. Microsoft has a strategy on waste manag...,3.0,NaN,directly,1,microsoft-sustainability-2022-112_0,True
3,15,2022 Microsoft Environmental Sustainability Re...,Does the company have a strategy on waste mana...,we recognize the urgent need to reduce carbon ...,to reach our commitment to become a zero waste...,37,text,[YES]. Microsoft has a strategy on waste manag...,3.0,NaN,directly,1,microsoft-sustainability-2022-104_1,True
4,14,2022 Microsoft Environmental Sustainability Re...,Does the company identify any impacts of its b...,our approach microsoft directly operates on ap...,microsoft directly operates on approximately 1...,46,text,"[YES]. Microsoft reports their emissions, land...",3.0,NaN,directly,1,microsoft-sustainability-2022-136_0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
423,3,Microsoft 2022 Environmental Sustainability Re...,Does the company have a specific process in pl...,the planetary computer aggregates and stores s...,the planetary computer aggregates and stores s...,63,text,YES. Microsoft has invested and participated i...,1.0,NaN,indirectly,1,microsoft-sustainability-2022-214_2,True
424,2,Microsoft 2022 Environmental Sustainability Re...,Does the company have a specific process in pl...,assessing climate risk most climate risk asses...,researchers at microsoft the universitat de va...,69,text,YES. Microsoft has invested and participated i...,1.0,NaN,indirectly,1,microsoft-sustainability-2022-239_3,True
425,1,Microsoft 2022 Environmental Sustainability Re...,Does the company have a specific process in pl...,assessing climate risk most climate risk asses...,the development and applications of emerging s...,69,text,YES. Microsoft has invested and participated i...,1.0,NaN,indirectly,1,microsoft-sustainability-2022-239_0,True
426,10,Microsoft 2022 Environmental Sustainability Re...,Does the company encourage downstream partners...,pursuing climate solutions through computation...,from this analysis researchers selected three ...,68,text,YES. Microsoft offers the Microsoft Planetary ...,1.0,NaN,indirectly,1,microsoft-sustainability-2022-234_1,True


In [ ]:
grouped_filtered_final = filtered.groupby(['Document', 'Question', 'Answer'], as_index=False).agg(lambda x: list(x))
grouped_filtered_final

,Document,Question,Answer,Context,Relevant,Page,Source From,Source Relevance Score,Unsure Flag,Addressed Directly,Core 16 Question,ParagraphID
0,Westpac 2023 climate report.pdf,Do the environmental/sustainability targets se...,Yes. Qantas's targets aligns with external cli...,[We recognise that human-induced climate chang...,[We are committed to supporting the aims of th...,[22],[text],[3.0],[nan],[indirectly],[1],[qantas-sustainability-2023-56_0]
1,Westpac 2023 climate report.pdf,Do the environmental/sustainability targets se...,Yes. Qantas's targets reference external clima...,[We recognise that human-induced climate chang...,[We are committed to supporting the aims of th...,[22],[text],[3.0],[nan],[indirectly],[1],[qantas-sustainability-2023-56_0]
2,Westpac 2023 climate report.pdf,Does the company have a strategy on waste mana...,Yes. Qantas has a strategy to achieve zero gen...,[We are focused on driving progress across thr...,[Operational and fleet efficiency: reducign si...,"[23, 36]","[text, text]","[3.0, 3.0]","[nan, nan]","[indirectly, indirectly]","[1, 1]","[qantas-sustainability-2023-61_2, qantas-susta..."
3,Westpac 2023 climate report.pdf,Does the company report short-term actions tak...,Yes. Qantas report actions planned to reduce w...,[We are focused on driving progress across thr...,[Operational and fleet efficiency: reducign si...,"[23, 36]","[text, text]","[3.0, 3.0]","[nan, nan]","[indirectly, indirectly]","[1, 1]","[qantas-sustainability-2023-61_2, qantas-susta..."


In [35]:
final_dataset = []
for ind, row in grouped_filtered_final.iterrows():
    paragraphs = [] 
    for p_id in row['ParagraphID']:
        paragraph_row = content_df[content_df['idx'] == p_id]
        paragraphs.append(
            {
                "title": paragraph_row['title'].iloc[0],
                "text": paragraph_row['text'].iloc[0],
                "is_supporting": True,
                "idx": paragraph_row['idx'].iloc[0]
            }
        )
        # print(paragraph_row) 
    final_dataset.append(
        {
            "id": ind,
            "question": row['Question'],
            "answer": row['Answer'],
            "answerable": True,
            "paragraphs": paragraphs
        }
    )

In [36]:
final_dataset

[{'id': 0,
  'question': 'Do the environmental/sustainability targets set by the company align with external climate change adaptation goals/targets?',
  'answer': "Yes. Qantas's targets aligns with external climate change adaptation targets.",
  'answerable': True,
  'paragraphs': [{'title': 'Climate action: OUR APPROACH',
    'text': 'We recognise that human-induced climate change is a significant issue for the aviation industry . We are committed to supporting the aims of the Paris Climate Agreement to limit warming to well below 2 * C above pre-industrial levels and pursuing efforts to limit the temperature increase to 1.5 * C above pre-industrial levels .',
    'is_supporting': True,
    'idx': 'qantas-sustainability-2023-56_0'}]},
 {'id': 1,
  'question': 'Do the environmental/sustainability targets set by the company reference external climate change adaptation goals/targets?',
  'answer': "Yes. Qantas's targets reference external climate change adaptation targets.",
  'answerab

In [37]:
save_json(root+org+orgs_code[orgs[IND]]+'.json', final_dataset)